**Problem Structure**
min c'x,  
s.t Ax ≤ b  
l ≤ x ≤ u.  

**Value of parameters**  
c = [8.6, −20.35, 61.5, 39.05, −0.15, 40.5, −0.05, −30.02, 40.65, 5.125]  

A= ([0,1,-2,0,1,-2,0,1,1,-1],  
    [-1,1,-3,2.5,0,1.4,0,-2,1,-1.5],  
    [-0.45,4.78,-2,-3.22,-1.3,1,-1,0,0,0.25],  
    [1,1,0,0,1,1,0,1,1,0] )  

b = [55.4, 67.9, 21.43, 12.7]

l = [−∞, −∞, 0, 0, −57, 0, −∞, −3, 4, 1],

u = [4, ∞, +∞, 2, 10, 45, 13, 20, +∞, +∞]



In [ ]:
!pip install pyomo

In [ ]:
!apt-get install -y -qq glpk-utils

In [3]:
from pyomo.environ import*
import numpy as np

#model creation
model=ConcreteModel()

# N is no. of variables and M is for no. of constraints
N=10
M=4

#objective function coefficients
obj_coef_c=np.array([8.6, -20.35, 61.5, 39.05, -0.15, 40.5, -0.05, -30.02, 40.65, 5.125])

#constraints coefficients
constr_coef_A=np.array([[0,1,-2,0,1,-2,0,1,1,-1],
                        [-1,1,-3,2.5,0,1.4,0,-2,1,-1.5],
                        [-0.45,4.78,-2,-3.22,-1.3,1,-1,0,0,0.25],
                        [1,1,0,0,1,1,0,1,1,0]])

#constraints rhs values
constr_rhs_b=np.array([55.4, 67.9, 21.43, 12.7])

#variables bounds
lower_bound=np.array([-np.inf, -np.inf, 0, 0, -57, 0, -np.inf, -3, 4, 1])
upper_bound=np.array([4, np.inf, np.inf, 2, 10, 45, 13, 20, np.inf, np.inf])

#list to create indices for traversing arrays
col_indices=np.arange(N)
row_indices=np.arange(M)

#printing content of list created
print(row_indices)
print(col_indices)
model.x=Var(col_indices)

model.constraints=ConstraintList()

#adding constraint to model
for i in row_indices:
    model.constraints.add(sum(constr_coef_A[i,j]*model.x[j] for j in col_indices)<=constr_rhs_b[i])

#setting bounds
for j in col_indices:
    model.x[j].setlb(lower_bound[j])
    model.x[j].setub(upper_bound[j])

#defining objective
model.objective=Objective(expr=sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

#printing model to check if it is correctly created
model.pprint()



[0 1 2 3]
[0 1 2 3 4 5 6 7 8 9]
2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :  -inf :  None :   inf : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 : -57.0 :  None :  10.0 : False :  True :  Reals
          5 :   0.0 :  None :  45.0 : False :  True :  Reals
          6 :  -inf :  None :  13.0 : False :  True :  Reals
          7 :  -3.0 :  None :  20.0 : False :  True :  Reals
          8 :   4.0 :  

**Model has been created properly as per the question.**

In [4]:
#solving model and printing solver status
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()



# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 11
  Number of nonzeros: 30
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.010608673095703125


In the solver status we see that  
  **Lower bound: -inf**  
  **Upper bound: inf** 

  this means that solver is unable to solve problem, the possible reason for this could be **problem is not bounded** or there is not a feasible solution.


In [5]:
#printing value of decision variables
print('\nvalue of decision variables')
for i in col_indices:
    print('x',i,'= ', model.x[i].value)


value of decision variables
x 0 =  None
x 1 =  None
x 2 =  None
x 3 =  None
x 4 =  None
x 5 =  None
x 6 =  None
x 7 =  None
x 8 =  None
x 9 =  None


**We can see that since solver is unable to optimise the problem, so none value has been assigned to the decision variables.**

**In this case if we try to print objective function or constraint values then we will get error, as they can not be calculated without assigning values to the variables.**

**Also it is not possibe to tell which constraints are active as they can not be computed**. 